In [ ]:
import logging
from pathlib import Path
import numpy as np
import geopandas as gpd
from shapely.geometry import Point, Polygon, MultiPolygon
from shapely.ops import unary_union
from scipy.spatial import ConvexHull
from sklearn.cluster import DBSCAN
import pandas as pd
import rasterio
import rasterio.mask
import matplotlib.pyplot as plt
from PySAGA_cmd import SAGA

In [44]:
saga_cmd_path = r"D:\Applications\saga\saga-9.10.2_x64\saga_cmd.exe"
saga = SAGA(saga_cmd_path)
release_raster = Path(r"../data/processed/simulations/sim_001/release_polygon_grid.sdat")
# output_dir = f"../data/processed/release_areas"
dem_path = Path(r"../data/processed/preprocessing/dem_filled.sdat")
grid_tools = saga / 'grid_tools'
resampling = grid_tools / "Resampling" 
release_grid = Path(r"../data/processed/simulations/sim_001/release_polygon_resample.sdat")

In [45]:
resampling.execute(
    INPUT=str(release_raster),
    TARGET_TEMPLATE=str(dem_path),
    OUTPUT=str(release_grid),        # Changed from RESULT to OUTPUT
    KEEP_TYPE=True,                  # Preserve integer data type
    SCALE_UP=0,                      # Nearest Neighbour for upscaling
    SCALE_DOWN=0,                    # Nearest Neighbour for downscaling
    TARGET_DEFINITION=1,             # 1 = use grid or grid system (template)
)

ExecutionError: A stderr was detected after the execution of "Resampling": 
ERROR 1: PROJ: proj_create: no database context specified
ERROR 1: Cannot parse CRS http://www.opengis.net/def/crs/EPSG/0/5482
ERROR 1: PROJ: proj_create: no database context specified
ERROR 1: Cannot parse CRS http://www.opengis.net/def/crs/EPSG/0/5936
ERROR 1: PROJ: proj_create: no database context specified
ERROR 1: Cannot parse CRS http://www.opengis.net/def/crs/EPSG/0/3978
ERROR 1: PROJ: proj_create: no database context specified
ERROR 1: Cannot parse CRS http://www.opengis.net/def/crs/EPSG/0/2193
ERROR 1: PROJ: proj_create: no database context specified
ERROR 1: Cannot parse CRS http://www.opengis.net/def/crs/EPSG/0/5482
ERROR 1: PROJ: proj_create: no database context specified
ERROR 1: Cannot parse CRS http://www.opengis.net/def/crs/EPSG/0/5936
ERROR 1: PROJ: proj_create: no database context specified
ERROR 1: Cannot parse CRS http://www.opengis.net/def/crs/EPSG/0/3978
ERROR 1: PROJ: proj_create: no data

In [46]:
gpp_lib = saga / 'sim_geomorphology' 
gpp_tool = gpp_lib / 0
process_area = Path(r"../data/processed/simulations/sim_001/process_area.sdat")
friction_model = 5
iterations=5
params = {
    'verbose': True,
    'DEM': str(dem_path),
    'RELEASE_AREAS': str(release_grid),
    'PROCESS_AREA': str(process_area),
    'PROCESS_PATH_MODEL': 1,           # 1 = Random Walk
    'RW_SLOPE_THRES': 40.0,            # Slope threshold for lateral spreading
    'RW_EXPONENT': 3.0,                # Exponent for lateral spreading (2.0-3.0 typical)
    'RW_PERSISTENCE': 1.5,             # Persistence factor (reduces abrupt direction changes)
    'GPP_ITERATIONS': iterations,
    'GPP_PROCESSING_ORDER': 2,         # 2 = RAs in parallel per iteration (recommended)
    'GPP_SEED': 1,                     # Random seed (1 = use current time)
    'FRICTION_MODEL': friction_model,
    'FRICTION_THRES_FREE_FALL': 60.0,  # Minimum slope for free fall
    'SINK_MIN_SLOPE': 2.5              # Minimum slope to preserve on sink filling
}
            

In [47]:
friction_mu=0.25
max_velocity=Path(r"../data/processed/simulations/sim_001/max_velocity.sdat")
stop_positions=Path(r"../data/processed/simulations/sim_001/stop_positions.sdat")
mass_to_drag=200
if friction_model == 4:
    params['FRICTION_MU'] = friction_mu
    params['FRICTION_METHOD_IMPACT'] = 0      # 0 = Energy Reduction
    params['FRICTION_IMPACT_REDUCTION'] = 75.0  # Energy reduction on impact (%)
    params['FRICTION_MODE_OF_MOTION'] = 0     # 0 = Sliding
    params['MAX_VELOCITY'] = str(max_velocity)

if friction_model == 5:
    params['FRICTION_MU'] = friction_mu
    params['FRICTION_MASS_TO_DRAG'] = mass_to_drag
    params['FRICTION_INIT_VELOCITY'] = 1.0
    params['MAX_VELOCITY'] = str(max_velocity)

params['STOP_POSITIONS'] = str(stop_positions)
gpp_tool.execute(**params)

-------------------------
sim_geomorphology / 0
    -DEM=..\data\processed\preprocessing\dem_filled.sdat -RELEASE_AREAS=..\data\processed\simulations\sim_001\release_polygon_resample.sdat -PROCESS_AREA=..\data\processed\simulations\sim_001\process_area.sdat -PROCESS_PATH_MODEL=1 -RW_SLOPE_THRES=40.0 -RW_EXPONENT=3.0 -RW_PERSISTENCE=1.5 -GPP_ITERATIONS=5 -GPP_PROCESSING_ORDER=2 -GPP_SEED=1 -FRICTION_MODEL=5 -FRICTION_THRES_FREE_FALL=60.0 -SINK_MIN_SLOPE=2.5 -FRICTION_MU=0.25 -FRICTION_MASS_TO_DRAG=200 -FRICTION_INIT_VELOCITY=1.0 -MAX_VELOCITY=..\data\processed\simulations\sim_001\max_velocity.sdat -STOP_POSITIONS=..\data\processed\simulations\sim_001\stop_positions.sdat

[████████████████████████████████████████████████████████████] 100/100% | Elapsed time: 0:19:09        


ExecutionError: A stderr was detected after the execution of "0": 
ERROR 1: PROJ: proj_create: no database context specified
ERROR 1: Cannot parse CRS http://www.opengis.net/def/crs/EPSG/0/5482
ERROR 1: PROJ: proj_create: no database context specified
ERROR 1: Cannot parse CRS http://www.opengis.net/def/crs/EPSG/0/5936
ERROR 1: PROJ: proj_create: no database context specified
ERROR 1: Cannot parse CRS http://www.opengis.net/def/crs/EPSG/0/3978
ERROR 1: PROJ: proj_create: no database context specified
ERROR 1: Cannot parse CRS http://www.opengis.net/def/crs/EPSG/0/2193
ERROR 1: PROJ: proj_create: no database context specified
ERROR 1: Cannot parse CRS http://www.opengis.net/def/crs/EPSG/0/5482
ERROR 1: PROJ: proj_create: no database context specified
ERROR 1: Cannot parse CRS http://www.opengis.net/def/crs/EPSG/0/5936
ERROR 1: PROJ: proj_create: no database context specified
ERROR 1: Cannot parse CRS http://www.opengis.net/def/crs/EPSG/0/3978
ERROR 1: PROJ: proj_create: no database cont

In [7]:
dem_path = Path(r"../data/processed/preprocessing/dem_filled.sdat")
dem_tif = Path(r"../data/processed/preprocessing/dem_filled.tif")

In [9]:
with rasterio.open(dem_path, 'r') as src:
    dem_data = src.read()
    out_meta = src.meta.copy()
    out_meta.update({
        'driver':'GTiff',
        'count': src.count,
        'dtype': dem_data.dtype,
        'nodata': src.nodata,
    })
    with rasterio.open(dem_tif, 'w', **out_meta) as dst:
        dst.write(dem_data)

In [ ]:
input_file = r'../data/raw/aoi/aoi.geojson'
gdf = gpd.read_file(input_file)

In [ ]:
gdf = gdf.set_crs("EPSG:31255")